# Compiling functions on vectors with Numba

First, [install](../index.md#installation) and import Vector and [Numba](https://numba.pydata.org/).

In [1]:
import vector
import numba as nb
import numpy as np

Numba is a just-in-time (JIT) compiler for a mathematically relevant subset of NumPy and Python. It allows you to write fast code without leaving the Python environment. The drawback of Numba is that it can only compile code blocks involving objects and functions that it recognizes.

The Vector library includes extensions to inform Numba about [vector objects](object.md), ~~[arrays of vectors](numpy.md)~~, and [arrays of Awkward Arrays](awkward.md). At the time of writing, the implementation of vector NumPy arrays is incomplete (see issue [#43](https://github.com/scikit-hep/vector/issues/43)).

Consider the following function:

In [2]:
@nb.njit
def compute_mass(v1, v2):
    return (v1 + v2).mass

In [3]:
compute_mass(vector.obj(px=1, py=2, pz=3, E=4), vector.obj(px=-1, py=-2, pz=-3, E=4))

8.0

When the two `MomentumObject4D` objects are passed as arguments, Numba recognizes them and replaces the Python objects with low-level structs. When it compiles the function, it recognizes `+` as the 4D `add` function and recognizes `.mass` as the `tau` component of the result.

Although this demonstrates that Numba can manipulate vector objects, there is no performance advantage (and a likely disadvantage) to compiling a calculation on just a few vectors. The advantage comes when many vectors are involved, in arrays.

In [4]:
# This is still not a large number. You want millions.
array = vector.Array(
    [
        [
            dict(
                {x: np.random.normal(0, 1) for x in ("px", "py", "pz")},
                E=np.random.normal(10, 1),
            )
            for inner in range(np.random.poisson(1.5))
        ]
        for outer in range(50)
    ]
)
array

<MomentumArray4D [[], ..., [{x: 0.0877, ...}, ...]] type='50 * var * Moment...'>

In [5]:
@nb.njit
def compute_masses(array):
    out = np.empty(len(array), np.float64)
    for i, event in enumerate(array):
        total = vector.obj(px=0.0, py=0.0, pz=0.0, E=0.0)
        for vec in event:
            total = total + vec
        out[i] = total.mass
    return out

In [6]:
compute_masses(array)

array([ 0.        , 29.59334472,  9.96825092, 29.98088102, 30.022318  ,
        8.77904697, 10.52123699,  9.30633562,  9.69685471,  7.90724598,
        8.22083106, 12.10506017,  9.14678916, 42.27902654, 10.08573923,
        9.98477624,  0.        , 22.79438116, 29.12919935, 32.07848403,
       29.6325778 ,  0.        ,  8.11447927, 39.07959905, 30.54027295,
       18.39609413, 21.15982485, 18.62455179,  7.75621961, 19.52526457,
       19.52907948, 20.40038164,  8.84074954,  9.18655937, 30.22076618,
        8.47538375, 10.42253874, 17.84485932,  0.        , 41.55064913,
        0.        , 10.60246245, 11.24522316, 19.32603825, 31.28879051,
       20.49852241,  0.        , 20.21304572,  0.        , 20.76651039])